In [1]:
# Load model directly
#8496
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd
# tokenizer = AutoTokenizer.from_pretrained("/work/frink/models/alpaca-13b")
# model = AutoModelForCausalLM.from_pretrained("/work/frink/models/alpaca-13b", torch_dtype=torch.bfloat16, 
#     device_map = "auto")
# model = model.to('cuda')
from tqdm import tqdm
import string

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

device = 'cuda'
class AlpacaInference():
    def __init__(self, alpaca_path):
        self.tokenizer = AutoTokenizer.from_pretrained(alpaca_path)
        self.model = AutoModelForCausalLM.from_pretrained(alpaca_path, torch_dtype=torch.bfloat16)
        self.model.to(device)

    
    def get_response(self, prompt):
        inputs = self.tokenizer(prompt, return_tensors="pt")
        input_ids = inputs.input_ids.to('cuda')
        generate_ids = self.model.generate(input_ids, max_length=input_ids.shape[-1] + 500 )
        # generate_ids.to(device)
        response = self.tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        return response
        


In [3]:
class AlpacBaselines():

    def __init__(self):
        self.alpaca_path = '/work/frink/models/alpaca-13b'
        self.alpaca_model = AlpacaInference(self.alpaca_path)

    def direct_assesment(self, source, summary, source_type = 'Dialogue', print_prompt = False):
        instruction = f'''Decide if the following summary is consistent with the corresponding {source_type.lower()}. Note that consistency means all information in the summary is supported by the {source_type.lower()}.\nAnswer(yes or no)'''
        prompt = f'{instruction}\n{source_type}: {source}\nSummary: {summary}:'
        if print_prompt:
            print(prompt)
        response = self.alpaca_model.get_response(prompt)
        response = response.split('Answer(yes or no):')[-1].strip().strip(string.punctuation)
        label = 1 if 'no' in response.lower() else 0
        return response, label





        

In [4]:


def get_alpaca_score(sources, summaries, eval_type, afacts = False):
    source_type = 'Dialogue'
    if afacts:
            source_type = 'Source'

    responses = []
    labels = []
    index = 0
    for src, summ in tqdm(list(zip(sources, summaries))):
        
        print_prompt = False
        if index < 1 or index%500 == 0:
                print_prompt = True
            
        if eval_type == 'direct_assesment':
            # print('here')
            response, label = alpaca_baselines.direct_assesment(src, 
                                                summ, 
                                                source_type = source_type, 
                                                print_prompt = print_prompt)
        
        index += 1
        responses.append(response)
        labels.append(label)
    return responses, labels



In [5]:
alpaca_baselines = AlpacBaselines()

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [6]:

read_path = '/home/ramprasad.sa/factual_evaluation_source_based/datasets/scored/xformer_llm_annotated_scored_label.csv'
write_path = '/home/ramprasad.sa/factual_evaluation_source_based/datasets/scored/xformer_llm_annotated_scored_label.csv'
df_scored = pd.read_csv(read_path)
df_scored.head()[:1]

print('WRITE PATH', write_path)

WRITE PATH /home/ramprasad.sa/factual_evaluation_source_based/datasets/scored/xformer_llm_annotated_scored_label.csv


In [7]:
df_scored.head()[:1]

,Unnamed: 0.1,Unnamed: 0,docid,model,nonfactual_spans,evidence,summary,factual_error,error_type,dialogue,origin,dialogue_atomic_facts,ChatGPT-direct_assesment_response,ChatGPT-direct_assesment_label,ChatGPT-direct_assesment_response_afacts,ChatGPT-direct_assesment_label_afacts,ChatGPT-direct_assessment_score_response,ChatGPT-direct_assessment_score_label,ChatGPT-direct_assessment_score_response_afacts,ChatGPT-direct_assessment_score_label_afacts
0,40,40,13681241,MV-BART,[],[],Callum is still busy and sorry.,0,[],Jair: Still busy?\r\nCallum: Yes a little sorr...,SAMSum,Jair asks if Callum is still busy.\nCallum con...,yes,0,Yes,0,100.0,100,100.0,100


In [8]:
def write_file(df, responses, labels, eval_type, afacts, write_path):
    column_response_key = f'Alpaca-{eval_type}_response'
    column_label_key = f'Alpaca-{eval_type}_label'
    if afacts:
        column_response_key = column_response_key + '_afacts'
        column_label_key = column_label_key + '_afacts'
    
    df[column_response_key] = responses
    df[column_label_key] = labels
    df.to_csv(write_path)

    return df

In [9]:
dialogues = list(df_scored['dialogue'].values)
summaries = list(df_scored['summary'].values)

eval_type = 'direct_assesment'    
afacts = False
responses, labels = get_alpaca_score(dialogues, 
                                     summaries, 
                                     eval_type = eval_type, 
                                     afacts = afacts)
df_scored = write_file(df_scored, responses, labels, eval_type, afacts, write_path)

  0%|                                                                                                                                                                                        | 0/1950 [00:00<?, ?it/s]

Decide if the following summary is consistent with the corresponding dialogue. Note that consistency means all information in the summary is supported by the dialogue.
Answer(yes or no)
Dialogue: Jair: Still busy?
Callum: Yes a little sorry
Jair: ok
Summary: Callum is still busy and sorry.:


 26%|████████████████████████████████████████████                                                                                                                                | 500/1950 [19:48<1:05:22,  2.71s/it]

Decide if the following summary is consistent with the corresponding dialogue. Note that consistency means all information in the summary is supported by the dialogue.
Answer(yes or no)
Dialogue: Drade: Mum he got out of the group chatting room
Marenda: I am sure he is angry!
Marenda: Why did you act that aggressively?
Marenda: You know your brother's temper
Drade: I just said what he had said was wrong!
Marenda: Apologize and send the invitation to him
Drade: Why should I?
Drade: I didn't say anything wrong!!!!!
Drade: He failed in his business, didn't he?
Marenda: But did you have to talk about it?
Drade: He acted like an asshole first
Marenda: Language! No need to talk about it more
Summary: Drade's brother got out of the group chatting room because he was angry. Drade didn't say anything wrong, he just said what he had said was wrong.:


 51%|████████████████████████████████████████████████████████████████████████████████████████▋                                                                                    | 1000/1950 [42:35<30:06,  1.90s/it]

Decide if the following summary is consistent with the corresponding dialogue. Note that consistency means all information in the summary is supported by the dialogue.
Answer(yes or no)
Dialogue: Daria: I got a new job
Kelsey: wooow where?
Daria: For a law firm! 
Kelsey: You go girl! 
Kelsey: So fast for a new graduate 
Kelsey: Well done!! 
Daria: Yeah!! 
Kelsey: Congrats!  
Daria: Thx
Summary: Daria informed Kelsey that she has secured a new job at a law firm, a swift achievement for a new graduate. Kelsey congratulated her and expressed enthusiasm and support for Daria''s accomplishment.:


 77%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                       | 1500/1950 [1:06:08<45:26,  6.06s/it]

Decide if the following summary is consistent with the corresponding dialogue. Note that consistency means all information in the summary is supported by the dialogue.
Answer(yes or no)
Dialogue: #Person1#: What does your sister look like, Mike?
#Person2#: Well, she's tall and pretty.
#Person1#: Is she like you?
#Person2#: I suppose so. We're both friendly and easy-going.
#Person1#: Is she as clever as you?
#Person2#: No, she's not as clever as me.
#Person1#: Big head!
Summary: Mike tells #Person1# his sister is tall and pretty and isn't as clever as he is.:


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1950/1950 [1:37:29<00:00,  3.00s/it]


In [11]:
df_scored

,Unnamed: 0.1,Unnamed: 0,docid,model,nonfactual_spans,evidence,summary,factual_error,error_type,dialogue,...,ChatGPT-direct_assesment_response,ChatGPT-direct_assesment_label,ChatGPT-direct_assesment_response_afacts,ChatGPT-direct_assesment_label_afacts,ChatGPT-direct_assessment_score_response,ChatGPT-direct_assessment_score_label,ChatGPT-direct_assessment_score_response_afacts,ChatGPT-direct_assessment_score_label_afacts,Alpaca-direct_assesment_response,Alpaca-direct_assesment_label
0,40,40,13681241,MV-BART,[],[],Callum is still busy and sorry.,0,[],Jair: Still busy?\r\nCallum: Yes a little sorr...,...,yes,0,Yes,0,100.0,100,100.0,100,Decide if the following summary is consistent ...,1
1,41,41,13681241,CODS,[],[],Callum is busy.,0,[],Jair: Still busy?\r\nCallum: Yes a little sorr...,...,Yes,0,Yes,0,100.0,100,100.0,100,Decide if the following summary is consistent ...,1
2,42,42,13681241,UniLM,[],[],Callum is still busy .,0,[],Jair: Still busy?\r\nCallum: Yes a little sorr...,...,Yes,0,Yes,0,100.0,100,100.0,100,Decide if the following summary is consistent ...,1
3,43,43,13681241,BART,[],[],Callum is busy.,0,[],Jair: Still busy?\r\nCallum: Yes a little sorr...,...,Yes,0,Yes,0,100.0,100,100.0,100,Decide if the following summary is consistent ...,1
4,44,44,13681241,gpt4-32k-0613,[],[],"Jair checks if Callum is still busy, and Callu...",0,[],Jair: Still busy?\r\nCallum: Yes a little sorr...,...,Yes,0,Yes,0,100.0,100,90.0,90,Decide if the following summary is consistent ...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1945,2023,2023,test_339,CODS,"[""and #Person1#'s mom "", '#Person1# and #Perso...",[],#Person1# and #Person2# are talking about what...,1,"['Intrinsic_Error', 'Intrinsic_Error']","#Person1#: Okay, next question. If Eric asked ...",...,Yes,0,Yes,0,90.0,90,60.0,60,Decide if the following summary is consistent ...,1
1946,2024,2024,test_339,UniLM,['person1 and person2 '],[],person1 and person2 are asking each other abou...,1,['Intrinsic_Error'],"#Person1#: Okay, next question. If Eric asked ...",...,Yes,0,Yes,0,80.0,80,70.0,70,Decide if the following summary is consistent ...,1
1947,2025,2025,test_339,BART,['##Person1# and #Person2# '],[],##Person1# and #Person2# are talking about wha...,1,['Intrinsic_Error'],"#Person1#: Okay, next question. If Eric asked ...",...,Yes,0,Yes,0,100.0,100,80.0,80,Decide if the following summary is consistent ...,1
1948,2026,2026,test_339,gpt4-32k-0613,[],[],Person1 and Person2 are asking each other hypo...,0,[],"#Person1#: Okay, next question. If Eric asked ...",...,yes,0,yes,0,100.0,100,100.0,100,Decide if the following summary is consistent ...,1


In [14]:
dialogues = list(df_scored['dialogue_atomic_facts'].values)
summaries = list(df_scored['summary'].values)

eval_type = 'direct_assesment'
afacts = True
responses, labels = get_alpaca_score(dialogues, 
                                     summaries, 
                                     eval_type = eval_type, 
                                     afacts = afacts)
df_scored = write_file(df_scored, responses, labels, eval_type, afacts, write_path)

  0%|                                                                                                                                                                                        | 0/1950 [00:00<?, ?it/s]

Decide if the following summary is consistent with the corresponding source. Note that consistency means all information in the summary is supported by the source.
Answer(yes or no)
Source: Jair asks if Callum is still busy.
Callum confirms that he is still busy.
Callum apologizes for being busy.
Jair acknowledges Callum's response.
Summary: Callum is still busy and sorry.:


 18%|███████████████████████████████                                                                                                                                             | 352/1950 [23:28<1:46:35,  4.00s/it]

KeyboardInterrupt



In [13]:
df_scored.keys()

Index(['Unnamed: 0.1', 'Unnamed: 0', 'docid', 'model', 'nonfactual_spans',
       'evidence', 'summary', 'factual_error', 'error_type', 'dialogue',
       'origin', 'dialogue_atomic_facts', 'ChatGPT-direct_assesment_response',
       'ChatGPT-direct_assesment_label',
       'ChatGPT-direct_assesment_response_afacts',
       'ChatGPT-direct_assesment_label_afacts',
       'ChatGPT-direct_assessment_score_response',
       'ChatGPT-direct_assessment_score_label',
       'ChatGPT-direct_assessment_score_response_afacts',
       'ChatGPT-direct_assessment_score_label_afacts',
       'Alpaca-direct_assesment_response', 'Alpaca-direct_assesment_label'],
      dtype='object')

In [12]:
set(responses)



{'1. Is Phil interested in technology? Yes\n2. Does Twitter have something for fashion? No\n3. Does Vic prefer Facebook? Yes\n4. Does Vic have an account on Tumblr? No\n5. Does Phil prefer Twitter? No\n6. Does Nancy have her reasons? Yes\n7. Does Phil have time to write a blog? No\n8. Does Vic have time to write a blog? No\n9. Does Twitter have news? Yes\n10. Does Instagram have news? Yes',
 "1. Is Phil's preference for Facebook consistent with the dialogue?\n2. Is Vic's preference for Twitter consistent with the dialogue?\n3. Is Vic's statement that he has never had an account on Tumblr consistent with the dialogue?\n4. Is Phil and Vic's statement that they don't have time to write a blog consistent with the dialogue?\n1. Yes, 2. Yes, 3. Yes, 4. Yes",
 '1. Is the summary consistent with the dialogue? Yes\n2. Does Vic prefer Facebook to Twitter? Yes\n3. Does Phil prefer Facebook to Twitter? Yes\n4. Does Vic have an account on Tumblr? No\n5. Does Phil have an account on Tumblr? No',
 '1